# 01A Data Cleansing Weather

## Imports

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

## Read Data

In [120]:
weather_df = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## One Hot Encode CodeSum Data

In [121]:
codes = {code: [0]*weather_df.shape[0] for codes in weather_df.CodeSum.unique() for code in codes.split()}

weather_df = weather_df.join(pd.DataFrame(codes))
for i, codesum in enumerate(weather_df.CodeSum):
    for code in codesum.split():
        weather_df.at[i, code] = 1
        
codes_df = weather_df.groupby('Date', as_index=True)[list(codes.keys())].max()

## Export codes_df

In [122]:
with open('../Cleansed_Data/codes_df.pkl', 'wb') as f:
    pickle.dump(codes_df, f)

----------

# New Weather Processing

## Import weather data

In [123]:
weather = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## Impute missing Tavg Values and Convert column to Int

In [124]:
missing_Tavg = weather[weather.Tavg == 'M'].index

weather.loc[missing_Tavg, 'Tavg'] = round((weather.loc[missing_Tavg, 'Tmax'] 
                                              + weather.loc[missing_Tavg, 'Tmin'])/2)
weather.Tavg = weather.Tavg.astype(int)

## Impute missing AvgSpeed and Convert column to float

In [125]:
median_wind = weather[weather.AvgSpeed!='M'].AvgSpeed.median()
weather.AvgSpeed.replace('M', median_wind, inplace=True)
weather.AvgSpeed = weather.AvgSpeed.astype(float)

## Create Temperature Columns in Celsius

In [126]:
def celsius(x):
    c = ((x - 32) * 5.0)/9.0
    return float(c)

In [127]:
weather['TavgC'] = weather['Tavg'].apply(celsius)
weather['TminC'] = weather['Tmin'].apply(celsius)
weather['TmaxC'] = weather['Tmax'].apply(celsius)

## Calculate Relative Humidity

In [128]:
weather['relative_humidity'] = 100 * (np.exp((17.625 * weather.DewPoint.apply(celsius)) / 
                                             (243.04 + weather.DewPoint.apply(celsius))) / 
                                      np.exp((17.625 * weather.Tavg.apply(celsius)) / 
                                             (243.04 + weather.Tavg.apply(celsius))))

## Convert DewPoint to Celsius

In [129]:
weather['DewPointC'] = weather['DewPoint'].apply(celsius)

## Add rain boolean

- TS THUNDERSTORM
- GR HAIL
- RA RAIN
- DZ DRIZZLE
- SH SHOWER

In [130]:
def filter_codesum(x):
    for code in rain:
        if code in x:
            return 1
        else:
            return 0

In [131]:
rain = ['RA', 'DZ', 'SH', 'TS', 'GR']

In [132]:
weather['rain'] = weather.CodeSum.apply(filter_codesum)

## Add Ideal Environment boolean

In [133]:
weather.loc[(weather['TavgC'] >= 10) & (weather['TavgC'] <= 35) & 
            (weather['relative_humidity'] <= 95) & (weather['relative_humidity'] >= 40) & 
            (weather['AvgSpeed'] <= 6), 
            'ideal_environment']=1

weather.ideal_environment.fillna(0, inplace=True)

## Add Ideal Feeding Conditions boolean

In [134]:
weather.loc[(weather['TavgC'] >= 15) & (weather['TavgC'] <= 30) & 
            (weather['relative_humidity'] >= 8) & (weather['relative_humidity'] <= 72) & 
            (weather['AvgSpeed'] <= 6), 
            'ideal_feeding']=1

weather.ideal_feeding.fillna(0, inplace=True)

## Average values to drop station

In [135]:
new_weather_df = weather.groupby('Date')['TavgC','TmaxC','TminC','DewPointC','relative_humidity',
                                         'rain','ideal_environment','ideal_feeding'].mean()

## Calculate Photoperiod

In [136]:
sun = weather[weather.Station == 1][['Date','Sunset','Sunrise']]

In [137]:
sun['sundiff'] = sun.apply(lambda x: int(x.Sunset) - int(x.Sunrise), axis=1)

In [138]:
sun['photoperiod'] =sun.sundiff.map(lambda x: int(str(x)[:2]) + int(str(x)[-2:])/60)

In [139]:
sun.index = sun.Date

In [140]:
sun = sun.drop(['Date','Sunset','Sunrise','sundiff'], axis=1)

## Join Photoperiod with new_weather_df

In [141]:
new_weather_df['Date'] = new_weather_df.index

In [142]:
new_weather_df = new_weather_df.join(sun, on='Date')

In [143]:
new_weather_df = new_weather_df.drop(['Date'], axis=1)

In [144]:
new_weather_df.head()

,TavgC,TmaxC,TminC,DewPointC,relative_humidity,rain,ideal_environment,ideal_feeding,photoperiod
Date,,,,,,,,,
2007-05-01,19.722222,28.611111,10.555556,10.555556,55.482501,0.0,0.0,0.0,14.016667
2007-05-02,10.833333,15.277778,5.833333,5.555556,69.905385,0.0,0.0,0.0,14.050000
2007-05-03,13.888889,19.166667,8.333333,4.444444,52.966288,0.0,0.0,0.0,14.083333
2007-05-04,16.111111,22.222222,10.000000,5.277778,48.837189,0.5,0.0,0.0,14.133333
2007-05-05,15.555556,18.888889,11.944444,3.611111,44.842896,0.0,0.0,0.0,14.166667


## Create Weather Features for Last 14 Days

In [145]:
def time_machine(df, column, days, bc = 0):
    new_df = df[[column]]
    for i in range(1,days+1):
        new_col_name = column + "_" + str(i) +"_back"
        new_col = one_day_back(df, column, i, bc)
        new_df[new_col_name] = new_col
    new_df.drop(column, 1, inplace=True)
    df = df.join(new_df)
    return df

def one_day_back(df, column, day, bc):
    col = df[column].tolist()
    for i in range(day):
        del col[-1]
        col.insert(0, bc)
    return col

In [146]:
new_weather_df.columns

Index(['TavgC', 'TmaxC', 'TminC', 'DewPointC', 'relative_humidity', 'rain',
       'ideal_environment', 'ideal_feeding', 'photoperiod'],
      dtype='object')

In [147]:
for col in new_weather_df.columns:
    new_weather_df = time_machine(new_weather_df, col, 14)


/Users/yibingchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yibingchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


## Export new_weather_df

In [148]:
with open('../Cleansed_Data/new_weather_df.pkl', 'wb') as f:
    pickle.dump(new_weather_df, f)